# Generate test data simulating calls in an emergency dispatch center

### Install NuGet-Packages +  Using-Statements

In [ ]:
#r "nuget:MathNet.Numerics"
#r "nuget:CsvHelper"

In [ ]:
using MathNet.Numerics.Distributions;
using XPlot.Plotly;

### Model class

In [ ]:
class Call 
{
    public DateTime Date { get; set; }
    
    public int Month { get; set; }
    
    public string DayOfWeek { get; set; }
        
    public int Hour { get; set; }
    
    public string WeatherConditions { get; set; }
}

### Generate random data for one day

In [ ]:
var random = new Random();

private IEnumerable<Call> GenerateDay(DateTime date)
{
    double callCount = 500+random.Next(100);
    
    // up to 20 % more calls between November and February
    if (date.Month >= 11 || date.Month <= 2)
    {
        callCount *= 1 + random.NextDouble() * 0.2;
    }
    
    string weatherConditions = random.NextDouble() > 0.7 ? "bad" : "good";
    
    // 40-60 % more calls when bad weather conditions occur
    if (weatherConditions == "bad")
    {
        callCount *= 1.4 + random.NextDouble() * 0.2;
    }
    
    var weekDayFactor = new Dictionary<DayOfWeek, float>()
    {
        { DayOfWeek.Monday, 1.4f },
        { DayOfWeek.Tuesday, 1.2f },
        { DayOfWeek.Wednesday, 1.0f },
        { DayOfWeek.Thursday, 1.2f },
        { DayOfWeek.Friday, 1.4f },
        { DayOfWeek.Saturday, 0.8f },
        { DayOfWeek.Sunday, 0.3f },
    };
    
    // consider day of week factor
    callCount *= weekDayFactor[date.DayOfWeek];
    
    Normal normalDist = new Normal(11, 3);
    var values = normalDist.Samples().Take((int)Math.Floor(callCount)).ToList();

    var calls = values.Select(v => new Call()
    {
        Date = date.Date,
        Month = date.Month,
        DayOfWeek = date.DayOfWeek.ToString(),
        Hour = (int)Math.Round(v),
        WeatherConditions = weatherConditions
    }).ToList();
    
    return calls;
}


### Generate one year

In [ ]:
DateTime currentDate = new DateTime(2019, 1, 1);
var calls = new List<Call>();

for (int i=0; i<365; i++) 
{
   calls.AddRange(GenerateDay(currentDate));
   currentDate = currentDate.AddDays(1);
}

calls.Count

### Visualize generated data

In [ ]:
using System.Linq;

var callsByDate = calls.GroupBy(c => c.Date).ToDictionary(c => c.Key, c => c.Count());

Chart.Plot(
    new Graph.Scatter() {
        x = callsByDate.Keys,
        y = callsByDate.Values
    }
)

### Group generated calls by hour

In [ ]:
using System.Linq;

var groupedByHour = (from c in calls
                     group c by new
                     {
                         c.Date,
                         c.Hour,
                         c.WeatherConditions
                     }
                     into grouped
                     orderby grouped.Key.Date, grouped.Key.Hour
                     select new
                     {
                         Date = grouped.Key.Date,
                         Hour = grouped.Key.Hour,
                         Month = grouped.Key.Date.Month,
                         DayOfWeek = grouped.Key.Date.DayOfWeek.ToString(),
                         WeatherConditions = grouped.Key.WeatherConditions,
                         Calls = grouped.Count()
                     }).ToList();

groupedByHour.Take(100)

### Export to CSV

In [ ]:
using CsvHelper;

using (var writer = new System.IO.StreamWriter("data\\calls.csv"))
using (var csv = new CsvWriter(writer, System.Globalization.CultureInfo.InvariantCulture))
{    
    csv.WriteRecords(groupedByHour);
}

### Add columns with data from previous hours

In [ ]:
var addPrevious = (from g in groupedByHour
                  let prev = groupedByHour.FirstOrDefault(p => p.Date == g.Date && p.Hour == g.Hour - 1)
                  let prev2 = groupedByHour.FirstOrDefault(p => p.Date == g.Date && p.Hour == g.Hour - 2)
                  select new
                  {
                      g.Date,
                      g.Hour,
                      g.Month,
                      g.DayOfWeek,
                      g.WeatherConditions,
                      g.Calls,
                      CallsPrevHour = prev?.Calls ?? 0,
                      CallsPrevHour2 = prev2?.Calls ?? 0
                  }).ToList();
  


### Export to CSV

In [ ]:
using CsvHelper;

using (var writer = new System.IO.StreamWriter("data\\calls-previous.csv"))
using (var csv = new CsvWriter(writer, System.Globalization.CultureInfo.InvariantCulture))
{    
    csv.WriteRecords(addPrevious);
}